Using matches_ddsm_CBIS_DDSM.csv file, we can get what ddsm is train or test using CBIS ddsm split.

All DDSM images that are not in CBIS_DDSM dataset are considered as training data (most of them are normal images).

In [4]:
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import pathlib

In [7]:
data_folder = pathlib.Path('/home/alalbiol/Data/mamo')
data_folder = pathlib.Path('/media/HD/mamo')
ddsm_path = data_folder / 'DDSM_png'

In [10]:
all_ddsm_images = list(ddsm_path.glob('**/*.png'))
print(f'DDSM images: {len(all_ddsm_images)}')


calc_case_test = pd.read_csv( '../resources/cbis_ddsm/calc_case_description_test_set.csv')
calc_case_train = pd.read_csv( '../resources/cbis_ddsm/calc_case_description_train_set.csv')
mass_case_test = pd.read_csv( '../resources/cbis_ddsm/mass_case_description_test_set.csv')
mass_case_train = pd.read_csv( '../resources/cbis_ddsm/mass_case_description_train_set.csv')
all_case_train = pd.concat([calc_case_train, mass_case_train])
all_case_test = pd.concat([calc_case_test, mass_case_test])

print(f"Number of train cbis abnormalities: {len(all_case_train)}")
print(f"Number of test cbis abnormalities: {len(all_case_test)}")

print(f"Number of all abnormalities: {len(all_case_train) + len(all_case_test)}")


DDSM images: 10420
Number of train cbis abnormalities: 2864
Number of test cbis abnormalities: 704
Number of all abnormalities: 3568


# Classify DDSM intro train/test

In [18]:
import pandas as pd

df_matches = pd.read_csv('../resources/matches_ddsm_CBIS_DDSM.csv')


def simplify_ddsm_name(row):
    return "/".join(row['ddsm_image'].split('/')[-4:])

def split_train_test(row):
    if row['cbis_image'] in all_case_train['image file path'].values:
        return 'train'
    elif row['cbis_image'] in all_case_test['image file path'].values:
        return 'test'
    else:
        return 'unknown'
    
df_matches['split'] = df_matches.apply(split_train_test, axis=1)
df_matches['ddsm_image'] = df_matches.apply(simplify_ddsm_name, axis=1)
    
print(df_matches.head())


                                          cbis_image  \
0  Calc-Training_P_00005_RIGHT_CC/1.3.6.1.4.1.959...   
1  Calc-Training_P_00005_RIGHT_MLO/1.3.6.1.4.1.95...   
2  Calc-Training_P_00007_LEFT_CC/1.3.6.1.4.1.9590...   
3  Calc-Training_P_00007_LEFT_MLO/1.3.6.1.4.1.959...   
4  Calc-Training_P_00008_LEFT_CC/1.3.6.1.4.1.9590...   

                                          ddsm_image  distance  split  
0   cancers/cancer_12/case4110/D_4110_1.RIGHT_CC.png  0.002982  train  
1  cancers/cancer_12/case4110/D_4110_1.RIGHT_MLO.png  0.005820  train  
2    benigns/benign_02/case1326/A_1326_1.LEFT_CC.png  0.005775  train  
3   benigns/benign_02/case1326/A_1326_1.LEFT_MLO.png  0.005494  train  
4  benign_without_callbacks/benign_without_callba...  0.009336  train  


In [19]:
df_matches['split'].value_counts()

split
train    2864
test      704
Name: count, dtype: int64

In [20]:
print("number of images ", len(df_matches.cbis_image.unique()))
print("number of images ", len(df_matches.ddsm_image.unique()))


number of images  3103
number of images  3032


El numero de imágenes de DDSM es menor porque en CBIS han repetido imagenes que tienen ambos tipos de lesiones con nombres diferentes.

In [21]:
print("df_matches ", df_matches.head())

df_matches                                            cbis_image  \
0  Calc-Training_P_00005_RIGHT_CC/1.3.6.1.4.1.959...   
1  Calc-Training_P_00005_RIGHT_MLO/1.3.6.1.4.1.95...   
2  Calc-Training_P_00007_LEFT_CC/1.3.6.1.4.1.9590...   
3  Calc-Training_P_00007_LEFT_MLO/1.3.6.1.4.1.959...   
4  Calc-Training_P_00008_LEFT_CC/1.3.6.1.4.1.9590...   

                                          ddsm_image  distance  split  
0   cancers/cancer_12/case4110/D_4110_1.RIGHT_CC.png  0.002982  train  
1  cancers/cancer_12/case4110/D_4110_1.RIGHT_MLO.png  0.005820  train  
2    benigns/benign_02/case1326/A_1326_1.LEFT_CC.png  0.005775  train  
3   benigns/benign_02/case1326/A_1326_1.LEFT_MLO.png  0.005494  train  
4  benign_without_callbacks/benign_without_callba...  0.009336  train  


In [26]:
all_ddsm_ids =["/".join(str(x).split('/')[-4:]) for x in all_ddsm_images]

print("all_ddsm_ids ", all_ddsm_ids[:5])

all_ddsm_df = pd.DataFrame(all_ddsm_ids, columns=['ddsm_image'])
all_ddsm_df['split'] = 'train'

all_ddsm_ids  ['cancers/cancer_13/case4124/D_4124_1.LEFT_CC.png', 'cancers/cancer_13/case4124/D_4124_1.RIGHT_CC.png', 'cancers/cancer_13/case4124/D_4124_1.RIGHT_MLO.png', 'cancers/cancer_13/case4124/D_4124_1.LEFT_MLO.png', 'cancers/cancer_13/case4161/D_4161_1.LEFT_CC.png']


In [27]:
for i, row in all_ddsm_df.iterrows():
    if row['ddsm_image'] in df_matches.ddsm_image.values:
        all_ddsm_df.loc[i, 'split'] = df_matches[df_matches['ddsm_image'] == row['ddsm_image']]['split'].values[0]

In [30]:
print(all_ddsm_df.split.value_counts())
print(df_matches.split.value_counts())
# De nuevo hay menos de test porque algunas de CBIS estan repetidas en masses y calcifications

split
train    9808
test      612
Name: count, dtype: int64
split
train    2864
test      704
Name: count, dtype: int64


In [34]:
all_ddsm_df.to_csv('../resources/ddsm/DDSM_split.csv', index=False)
# now save train and test images in different csv files
all_ddsm_df[all_ddsm_df['split'] == 'train'].to_csv('../resources/ddsm/DDSM_train.csv', index=False)
all_ddsm_df[all_ddsm_df['split'] == 'test'].to_csv('../resources/ddsm/DDSM_test.csv', index=False)